# Kalman-Filter Elemente

Verwenden sie die Nomenklautur aus der Literatur (P,F,x,H,y,K,...)

In [1]:
import numpy as np
np.set_printoptions(precision=5)
np.set_printoptions(suppress=True)

In [2]:
P = np.matrix([[np.square(0.1), 0, 0, 0], \
               [0, np.square(0.1), 0, 0], \
               [0, 0, np.square(0.1), 0], \
               [0, 0, 0, np.square(0.1)]])

R = np.matrix([[np.square(0.05), 0, 0], \
               [0, np.square(0.05), 0], \
               [0, 0, np.square(0.1)]])

W = np.matrix([[np.square(0.05), 0, 0, 0], \
               [0, np.square(0.05), 0, 0], \
               [0, 0, np.square(0.1), 0], \
               [0, 0, 0, np.square(0.1)]])

# 1.a State-Vector Prior

```txt
Der Zustandsvektor beschreibt:
x_kp1   - Aktuelle x Koordinate im Raum           [m]       (glober Bezug)
y_kp1   - Aktuelle y Koordinate im Raum           [m]       (glober Bezug)
vm_kp1  - Aktuelle Geschwindigkeit im Raum        [m/s]     (glober Bezug)
psi_kp1 - Aktuelle Richtung im Raum               [radiant] (glober Bezug)
```
Im Bezug zum state-vektor [aktuellen Position, Heading, Geschwindigkeit (Odometrie)] soll ein prior abgeschätzt werden. Nachfolgend werden zunächst die Werte der 0. Epoche gesetzt. Die berechneten Werte der 1. Epoche müssen mit den ..._solution Werten zusammenpassen

In [3]:
# Die Werte des 0. Zustandes 
x_k   = 1.6986523744
y_k   = -0.8705878426
vm_k  = 0.3689004346
psi_k = -136.9055361964

x_state = np.matrix([[x_k, y_k, vm_k, psi_k]]) 
### Aus ROS-Topics Messages (TWIST)
v_mean = 0.3808133602142334 # mittlere Geschwindigkeit [m/s] x-Achse (nach vorne)
rz_mean = -0.05285670161    # mittlere Drehgeschwindigkeit z-achse (drehen)
delta_t = 0.1524369716644287 #Zeitdifferenz Epoche 0 und Epoche 1 [sekunden]

### Hier die Berechnung des prior einfügen (Folie 7)
########################################################################################################
### Wie R_kp1 berechnet wird steht in Folie 6, siehe oben, stellen sie um.
R_kp1 = 0

########################################################################################################
delta_a = rz_mean * delta_t
R_ = (v_mean * delta_t)/delta_a
if (np.absolute(R_) < 0.001):
    print("R_ was too low")
    R_ = 0.001
print("R_")
print(R_)
#___xk+1 und yk+1___
coord_xy    = np.matrix([[x_state.item(0)]        , [x_state.item(1)]])
psi_rot     = np.matrix([[np.cos(x_state.item(3)) , -1*np.sin(x_state.item(3))], \
                         [np.sin(x_state.item(3)) , np.cos(x_state.item(3))]])
delta_a_rot = np.matrix([[np.sin( delta_a )]      , [1 - np.cos ( delta_a )]])
coord_xy_   = np.add(coord_xy, ( (R_ * psi_rot) * delta_a_rot ) )
#___vm,k+1___
v_mean_ = v_mean
#___psi_k+1___
# Kann ggf. gestützt werden durch Richtungsschätzung nach geradeausfahrt.
psi_ = x_state.item(3) + delta_a

x_state_ = np.matrix([coord_xy_.item(0), coord_xy_.item(1), v_mean_, psi_])
print("x_state_ : " , x_state_)
########################################################################################################

###
### In den nachfolgenden Array die Ergebnisse einfügen
#######___x_state_ = [0,0,0,0]
########################################################################################################


### CHECK RESULTS
x_state_solution = [1.713030141,-0.8143466814,0.3808133602,-136.9135935119]
x_state_sucess = np.allclose(x_state_, x_state_solution, rtol=1e-05, atol=1e-08, equal_nan=False)
if(x_state_sucess):
    print("x_state_ has the right result")
else:
    print("There is a difference in the results, please compare:")
    print(x_state_)
    print(x_state_solution)

R_
-7.204637228861572
x_state_ :  [[   1.71303   -0.81435    0.38081 -136.91359]]
x_state_ has the right result


# 1.b Kovarianzmatrix Prior

```txt
Die Kovarianzmatrix des Zustandsvektors beschreibt auf der Hauptdiagonalen die Varianz/Standardabweichung:
x_kp1   - Aktuelle x Koordinate im Raum           [m]       (glober Bezug)
y_kp1   - Aktuelle y Koordinate im Raum           [m]       (glober Bezug)
vm_kp1  - Aktuelle Geschwindigkeit im Raum        [m/s]     (glober Bezug)
psi_kp1 - Aktuelle Richtung im Raum               [radiant] (glober Bezug)
sowie die dazu gehörigen Kovarianzen auf den Nebendiagonalen (Korrelationen)
```

Das aufstellen der C-Matrix und der Kovarianzabschätzung W entfällt, fürs Verständis ist jedoch trotzdem die Auseinandersetzung damit wichtig.

Passen sie die Ableitungsmatrix F an. Hierfür wird (laut Folien) delta_t, psi_k, vm_kp1 und R_kp1 benötigt)


In [4]:
# Die Werte des 0. Zustandes 
x_k   = 1.6986523744
y_k   = -0.8705878426
vm_k  = 0.3689004346
psi_k = -136.9055361964

### Aus ROS-Topics Messages (TWIST)
v_mean = 0.3808133602142334 # mittlere Geschwindigkeit [m/s] x-Achse (nach vorne)
rz_mean = -0.05285670161    # mittlere Drehgeschwindigkeit z-achse (drehen)
delta_t = 0.1524369716644287 #Zeitdifferenz Epoche 0 und Epoche 1 [sekunden]

x_state = np.matrix([[x_k, y_k, vm_k, psi_k]]) 

### Die C-Matrix kann zunächst so stehen bleiben
C = np.matrix([[(np.square(delta_t)/2), 0,  0, 0], \
               [0, (np.square(delta_t)/2),  0, 0], \
               [0, 0,  delta_t, 0], \
               [0, 0,  0, delta_t]])

### Die P-Matrix der Epoche 0
P = np.matrix([[ 0.0004496854,  0.0000235108,  0.0000840152, -0.0007350205], \
               [ 0.0000235108,  0.0004801959,  0.0003926916,  0.0000673105], \
               [ 0.0000840152,  0.0003926916,  0.0010642684, -0.0000465955], \
               [-0.0007350205,  0.0000673105, -0.0000465955,  0.0026009225]])

### F muss angepasst werden, siehe Folie 8,9,10 
########################################################################################################

### Wie R_kp1 berechnet wird steht in Folie 6, siehe oben, stellen sie um.
#######___R_kp1 = 0
#######___F = np.matrix([[1, 0,  0, 0], \
#######___               [0, 1,  0, 0], \
#######___               [0, 0,  1, 0], \
#######___               [0, 0, 0,  1]])
########################################################################################################
R_ = (v_mean * delta_t)/delta_a
if (np.absolute(R_) < 0.001):
    print("R_ was too low")
    R_ = 0.001
print("R_")
print(R_)
delta_a = rz_mean * delta_t
psi_ = x_state.item(3) + delta_a
psik = x_state.item(3)
ox1_vk = delta_t * np.cos ( psi_ )
#print("ox1_vk")
#print(ox1_vk)

ox1_ypsk = R_ * np.cos ( psi_) - (R_ * np.cos(psik))
#print("ox1_ypsk")
#print(ox1_ypsk)

oy1_vk = delta_t * np.sin ( psi_ )
#print("oy1_vk")
#print(oy1_vk)

oy1_ypsk = R_ * np.sin ( psi_ ) - (R_ * np.sin ( psik ))
#print("oy1_ypsk")
#print(oy1_ypsk)

oyps_vk = delta_t / R_
#print("oyps_vk")
#print(oyps_vk)

#print("P")
#print(P)
F = np.matrix([[1, 0,  ox1_vk, ox1_ypsk], \
               [0, 1,  oy1_vk, oy1_ypsk], \
               [0, 0,       1,        0], \
               [0, 0, oyps_vk,        1]])

print(R_)
########################################################################################################

########################################################################################################

Pd = F*P*np.transpose(F)+C*W*np.transpose(C)


### CHECK RESULTS
Pd_solution = np.matrix([[ 0.0005491368,  0.0000408914,  0.0001274506, -0.000885783 ], \
                         [ 0.0000408914,  0.0006218449,  0.0005490375,  0.0000862149], \
                         [ 0.0001274506,  0.0005490375,  0.0012966387, -0.0000691135], \
                         [-0.000885783 ,  0.0000862149, -0.0000691135,  0.002835741 ]])
Pd_state_sucess = np.allclose(Pd, Pd_solution, rtol=1e-05, atol=1e-08, equal_nan=False)
if(Pd_state_sucess):
    print("Pd has the right result")
else:
    print("There is a difference in the results, please compare:")
    print(Pd)
    print(Pd_solution)

R_
-7.204637228861572
-7.204637228861572
Pd has the right result


# 2.a Innovation/Residuen y

```txt
Der Beobachtungsvektor z hat eine Länge von 3

Während die x- und y-Koordinaten direkt vom Tachymeter gemessen wird, wird die Geschwindigkeit aus der 
Distanzdifferenz zum vorhergegangenen Punkt und der vergangenen Zeit berechnet (hohe Korrelation, aber für implementationszwecke sinnvoll).

Der dazugehörige Raum des Zustandsvektors H*x haben ebenfalls die Länge von 3.

Der Innovationsvektor y beschreibt demnach die Differenz von der: 
x-Koordinate
y-Koordinate 
Geschwindigkeit

In der Differenz sollte die Innovation (y) möglichst gering ausfallen.

```

In [6]:
### Aus ROS-Topics Messages (PointStamped)
position_tachy_x =   1.7224763447
position_tachy_y =  -0.8316840883
tachy_velocity =    0.4069916495 # [m/s] Berechnet aus der vorangegangen Position und vergangener Zeit.

### Der prior Zustandsvektor x (siehe auch Aufgabe 1.a)
x_state_ = np.matrix([[1.713030141,-0.8143466814,0.3808133602,-136.9135935119]])

H = np.matrix([[1, 0, 0, 0], \
               [0, 1, 0, 0], \
               [0, 0, 1, 0]])
########################################################################################################
# y = z - H*x in python ausprogrammieren

########################################################################################################

z = np.matrix([[position_tachy_x,position_tachy_y,tachy_velocity]])
xd_beob_ = H*np.transpose(x_state_) 


y = np.subtract(np.transpose(z), xd_beob_)
y = np.transpose(y)
print(y)
########################################################################################################
########################################################################################################

### CHECK RESULTS
y_solution = np.matrix([[0.0094462037,-0.0173374069,0.0261782893]])
y_sucess = np.allclose(y, y_solution, rtol=1e-05, atol=1e-08, equal_nan=False)
if(y_sucess):
    print("y has the right result")
else:
    print("There is a difference in the results, please compare:")
    print(y)
    print(y_solution)


[[ 0.00945 -0.01734  0.02618]]
y has the right result


# 2.b K - Kalman-Gain

```txt
Der Zustandsvektor beschreibt:
x_kp1   - Aktuelle x Koordinate im Raum           [m]       (glober Bezug)
y_kp1   - Aktuelle y Koordinate im Raum           [m]       (glober Bezug)
vm_kp1  - Aktuelle Geschwindigkeit im Raum        [m/s]     (glober Bezug)
psi_kp1 - Aktuelle Richtung im Raum               [radiant] (glober Bezug)
```

In [7]:
# K ist eine Matrixenberechnung der größen Pd, H und R(Kovarianzmatrix)
# Pd ist auch das Ergebnis von Aufgabe 1.b
# Zum invertieren können sie die numpy (np) Funktion np.linalg.inv(...) verwenden.

# Berechnen sie das P-Update anhand der Formeln aus den Folien (Folie 16, Schur-Frobenius).
# Oder auch Anhand der Formeln in der Literatur. Vergleichen sie!
# Sigma xx (mit Strichen darüber) = Pd (siehe 4,4 Kovarianzmatrix oben)
# Sigma ll = R (siehe 3,3 Kovarianzmatrix)

Pd = np.matrix([[ 0.0005491368,  0.0000408914,  0.0001274506, -0.000885783 ], \
                [ 0.0000408914,  0.0006218449,  0.0005490375,  0.0000862149], \
                [ 0.0001274506,  0.0005490375,  0.0012966387, -0.0000691135], \
                [-0.000885783 ,  0.0000862149, -0.0000691135,  0.002835741 ]])

R = np.matrix([[np.square(0.05), 0, 0], \
               [0, np.square(0.05), 0], \
               [0, 0, np.square(0.1)]])

H = np.matrix([[1, 0, 0, 0], \
               [0, 1, 0, 0], \
               [0, 0, 1, 0]])

K = (Pd*np.transpose(H))*np.linalg.inv((H*Pd*np.transpose(H)) + R)

### CHECK RESULTS
K_solution = np.matrix([[ 0.1796042881,  0.0091967271,  0.0088088693], \
                        [ 0.0091967271,  0.1921843863,  0.0391575717], \
                        [ 0.0352354773,  0.1566302869,  0.1067708801], \
                        [-0.2907506588,  0.0321993374, -0.0044027042]])

K_sucess = np.allclose(K, K_solution, rtol=1e-05, atol=1e-08, equal_nan=False)
if(K_sucess):
    print("K has the right result")
else:
    print("There is a difference in the results, please compare:")
    print(K)
    print(K_solution)

[[ 0.1796   0.0092   0.00881]
 [ 0.0092   0.19218  0.03916]
 [ 0.03524  0.15663  0.10677]
 [-0.29075  0.0322  -0.0044 ]]
[[ 0.1796   0.0092   0.00881]
 [ 0.0092   0.19218  0.03916]
 [ 0.03524  0.15663  0.10677]
 [-0.29075  0.0322  -0.0044 ]]
K has the right result


# 3.a State-Vektor Update

```txt
Der Zustandsvektor beschreibt:
x_kp1   - Aktuelle x Koordinate im Raum           [m]       (glober Bezug)
y_kp1   - Aktuelle y Koordinate im Raum           [m]       (glober Bezug)
vm_kp1  - Aktuelle Geschwindigkeit im Raum        [m/s]     (glober Bezug)
psi_kp1 - Aktuelle Richtung im Raum               [radiant] (glober Bezug)
```

In [9]:
### Berechnen sie aus dem Kalman Gain K und der Innovation y den Zuschlag zum state-vektor (update)
# x = x + K*y

# Für K siehe auch Aufgabe 2.b
K = np.matrix([[ 0.1796042881,  0.0091967271,  0.0088088693], \
               [ 0.0091967271,  0.1921843863,  0.0391575717], \
               [ 0.0352354773,  0.1566302869,  0.1067708801], \
               [-0.2907506588,  0.0321993374, -0.0044027042]])

#Für y siehe auch Aufgabe 2.a
y = np.matrix([[0.0094462037,-0.0173374069,0.0261782893]])

# Für x_state_ siehe auch Aufgabe 1.a
x_state_ = np.matrix([[1.713030141,-0.8143466814,0.3808133602,-136.9135935119]])

#START
########################################################################################################

x_state_update = np.transpose(x_state_) + K * np.transpose(y)
x_state_update = np.transpose(x_state_update)








########################################################################################################
#ENDE


### CHECK RESULTS
x_state_update_solution = np.matrix([[ 1.7147978734,-0.8165667079,0.3812257177,-136.9170135101]])

x_state_update_sucess = np.allclose(x_state_update, x_state_update_solution, rtol=1e-05, atol=1e-08, equal_nan=False)
if(x_state_update_sucess):
    print("x_state_update has the right result")
else:
    print("There is a difference in the results, please compare:")
    print(x_state_update)
    print(x_state_update_solution)
print()


x_state_update has the right result



# 3.b Kovarianzmatrix Update

```txt
Verwenden sie nicht die Berechnungsformel aus der Literatur, diese ist nur für lineare Aufgaben gültig.
Berechnen sie das P-Update anhand der Formeln aus den Folien (Folie 16, Schur-Frobenius).
Sigma xx (mit Strichen darüber) = Pd (siehe 4,4 Kovarianzmatrix oben)
Sigma ll = R (siehe 3,3 Kovarianzmatrix)
```

In [10]:
Pd = np.matrix([[ 0.0005491368,  0.0000408914,  0.0001274506, -0.000885783 ], \
                [ 0.0000408914,  0.0006218449,  0.0005490375,  0.0000862149], \
                [ 0.0001274506,  0.0005490375,  0.0012966387, -0.0000691135], \
                [-0.000885783 ,  0.0000862149, -0.0000691135,  0.002835741 ]])

R = np.matrix([[np.square(0.05), 0, 0], \
               [0, np.square(0.05), 0], \
               [0, 0, np.square(0.1)]])

H = np.matrix([[1, 0, 0, 0], \
               [0, 1, 0, 0], \
               [0, 0, 1, 0]])

K = np.matrix([[ 0.1796042881,  0.0091967271,  0.0088088693], \
               [ 0.0091967271,  0.1921843863,  0.0391575717], \
               [ 0.0352354773,  0.1566302869,  0.1067708801], \
               [-0.2907506588,  0.0321993374, -0.0044027042]])

#START
########################################################################################################

P_update = Pd - (K * (R+H*Pd*np.transpose(H)) * np.transpose(K))








########################################################################################################
#ENDE



### CHECK RESULTS
P_update_solution = np.matrix([[ 0.0004490107,  0.0000229918,  0.0000880887, -0.0007268766], \
                               [ 0.0000229918,  0.000480461 ,  0.0003915757,  0.0000804983], \
                               [ 0.0000880887,  0.0003915757,  0.0010677088, -0.000044027 ], \
                               [-0.0007268766,  0.0000804983, -0.000044027 ,  0.0025751186]])

P_update_success = np.allclose(P_update, P_update_solution, rtol=1e-05, atol=1e-08, equal_nan=False)
if(P_update_success):
    print("P_update has the right result")
else:
    print("There is a difference in the results, please compare:")
    print(P_update)
    print(P_update_solution)

P_update has the right result
